In [ ]:
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
import scala.collection.JavaConversions._
import scala.util.control.Breaks._
import java.nio.file.{Files, Path, Paths}

object Helpers {
    def using[A <: { def close(): Unit }, B](resource: A)(f: A => B): B =
        try {
            f(resource)
        } finally {
            resource.close()
        }
}

import Helpers._

val en : String = "en"
val de : String = "de"
val fr : String = "fr"
val langs : List[String] = List(en, de, fr)

def detectLang(line : String, dicts : scala.collection.mutable.Map[String, List[String]]) : Option[String] = {
    val langs = line.split(" ").flatMap(item => dicts.filter(_._2.exists(_.equalsIgnoreCase(item))).map(_._1))
                    .groupBy(f => f)
                    .map(g => (g._1, g._2.size))
    if(langs.isEmpty) None
    else Some(langs.maxBy(_._2)._1)
}

val stopwordsPerLang: scala.collection.mutable.Map[String, List[String]] = scala.collection.mutable.Map.empty 
// Stopwords per each lang, are all stored in RAM, because used for all supported languages detection & they're relatively small, quick to read.  

for(lang <- langs) { // main loop. TODO: refactor into function and call it async
  using(scala.io.Source.fromFile(s"notebooks/words/vocabs/$lang-stopwords.txt")) { source => 
    for (line <- source.getLines) {
      val list = stopwordsPerLang.getOrElse(lang, List.empty)
      stopwordsPerLang.update(lang, list:+line)  
    }
  }
}

val textfilesPathsPerLang: scala.collection.mutable.Map[String, List[String]] = scala.collection.mutable.Map.empty
val textfiles = Files.newDirectoryStream(Paths.get("notebooks/words/text-data"))
  .filter(_.getFileName.toString.endsWith(".txt"))

for(filePath <- textfiles) {
  val file = filePath.toString
  using(scala.io.Source.fromFile(file)) { source => 
    val firstLine = source.getLines.next() // get single first line & detect language with it, TODO: better to use a few random lines in the middle of the text
    detectLang(firstLine, stopwordsPerLang) match  {
      case Some(lang) => {
        var list = textfilesPathsPerLang.getOrElse(lang, List.empty)
        textfilesPathsPerLang.update(lang, list:+file)        
      }
      case None => println("Language was not detected for file: $file")
    }                                              
  }
}

println(textfilesPathsPerLang) // TODO: for each language, iterate through it. may be async


case class Lemma(entry:String, default:String, pos:String)
object Lemma {
  def parse(line:String): Option[Lemma] = {
    line.split("\t") match { 
      case Array(f,s,t,_*) => Some(Lemma(f, s, t))
      case _ => None
    }
  }  
}

// TODO: lemmas per lang : fix
//val lemmasPerLang: scala.collection.mutable.Map[String, List[Lemma]] = scala.collection.mutable.Map.empty
for(lang <- langs) { // main loop. TODO: refactor into function and call it async
  
  var lemmas : List[Lemma] = List.empty
  /*
  using(scala.io.Source.fromFile(s"notebooks/words/vocabs/$lang-lemmatizer.txt")) { source => 
    for (line <- source.getLines) {
      val list = lemmasPerLang.getOrElse(lang, List.empty)
      Lemma.parse(line) match { 
        case Some(lemma) => lemmasPerLang.update(lang, list:+lemma)
        case _ =>;
      }      
    }
  }*/
//for(item <- lemmasPerLang.keys) println(item)
//for (item <- stopwordsPerLang.keys ) println(item)
}


Map(en -> List(notebooks/words/text-data/eng3.txt, notebooks/words/text-data/eng2.txt, notebooks/words/text-data/eng1.txt), fr -> List(notebooks/words/text-data/fr1.txt, notebooks/words/text-data/fr2.txt, notebooks/words/text-data/fr3.txt), de -> List(notebooks/words/text-data/de3.txt, notebooks/words/text-data/de4.txt, notebooks/words/text-data/de1.txt, notebooks/words/text-data/de2.txt))
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
import scala.collection.JavaConversions._
import scala.util.control.Breaks._
import java.nio.file.{Files, Path, Paths}
defined object Helpers
import Helpers._
en: String = en
de: String = de
fr: String = fr
langs: List[String] = List(en, de, fr)
detectLang: (line: String, dicts: scala.collection.mutable.Map[String,List[String]])Option[String]
stopwordsPerLang: scala.collection.mutable.Map[String,List[String]] = Map(en -> List(a, about, above, after, again, against, all, am, an, and, any, are, aren't, as, at